In [5]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y

--2021-12-22 15:18:16--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M   239MB/s    in 0.3s    

2021-12-22 15:18:16 (239 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::sqlite==3.31.1=h7b6447c_0
  - defaults/linux-64::numpy==1.21.2=py37h20f2e39_0
  - defaults/linux-64::pycparser==2.19=py37_0
  - defaults/linux-64::pand

In [6]:
# 导入包
import torch
import torch.optim as optim
from torch import nn as nn
import torch.nn.functional as F
from torch import sigmoid
from torch.autograd import Variable
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from rdkit.Chem import MACCSkeys

In [9]:
class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2215, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 2) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = sigmoid(self.fc4(h3))
        return output



def torch_mlp_reg(train_file, test_file):
  # process train data
  train_data = pd.read_csv(train_file)
  train_data_x = train_data['mol']
  train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
  train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
  train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)

  train_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in train_mols]
  train_maccs_fps_array = np.asarray(train_maccs_fps, dtype=float)

  train_fps_array = np.concatenate([train_morgan_fps_array, train_maccs_fps_array],axis=1)



  train_x_tensor = torch.from_numpy(train_fps_array)
  train_y = train_data['Class']
  train_y_tensor = torch.from_numpy(np.array(train_y))

  # process test data
  test_data = pd.read_csv(test_file)
  test_data_x = test_data['mol']
  test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
  test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
  test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)

  test_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in test_mols]
  test_maccs_fps_array = np.asarray(test_maccs_fps, dtype=float)

  test_fps_array = np.concatenate([test_morgan_fps_array, test_maccs_fps_array],axis=1)


  test_x_tensor = torch.from_numpy(test_fps_array)
  test_y = test_data['Class']
  test_y_tensor = torch.from_numpy(np.array(test_y))
  # model training
  epochs_list = [50, 80, 100, 130, 150, 170, 190, 200, 210, 230, 250]
  res_list = []
  for epochs in epochs_list:   
    #criterion = nn.MSELoss()
    model = QSAR_mlp()
    #losses = []
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    for epoch in range(epochs):
      data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).long()
      optimizer.zero_grad()
      y_pred = model(data)
      loss = F.cross_entropy(y_pred, target)
      #loss = criterion(y_pred.reshape(-1), target)
      #losses.append(loss.item())
      #if epoch % 20 == 0:
        #print("Loss {}: {}".format(epoch, loss.item()))
      loss.backward()
      optimizer.step()

    pred_test_y_prob = model(Variable(test_x_tensor).float()).tolist()

    torch_test_pred_list = []
    for i, test_prob in enumerate(pred_test_y_prob):
        pred_pos_prob = test_prob[1]
        torch_test_pred_list.append(round(pred_pos_prob, 3))

    torch_test_auc = round(roc_auc_score(test_y, torch_test_pred_list),2)
    torch_test_acc = round(accuracy_score(test_y, np.round(torch_test_pred_list)),2)
    torch_test_sen = round(recall_score(test_y, np.round(torch_test_pred_list)),2)
    torch_test_spc = round((torch_test_acc * len(test_y)- torch_test_sen * test_y.sum())/(len(test_y)-test_y.sum()),2)

    pred_train_y_prob = model(Variable(train_x_tensor).float()).tolist()

    torch_pred_list = []
    for i, train_prob in enumerate(pred_train_y_prob):
        pred_pos_prob = train_prob[1]
        torch_pred_list.append(round(pred_pos_prob, 3))

    torch_train_auc = round(roc_auc_score(train_y, torch_pred_list),2)
    torch_train_acc = round(accuracy_score(train_y, np.round(torch_pred_list)),2)
    torch_train_sen = round(recall_score(train_y, np.round(torch_pred_list)),2)
    torch_train_spc = round((torch_train_acc * len(train_y)- torch_train_sen * train_y.sum())/(len(train_y)-train_y.sum()),2)
    torch_perf = {'AUC':[torch_train_auc, torch_test_auc],
                'ACC':[torch_train_acc, torch_test_acc],
                'SEN':[torch_train_sen, torch_test_sen],
                'SPC':[torch_train_spc, torch_test_spc]}
    result = pd.DataFrame(torch_perf, index=['train', 'test'])
    res_list.append(result)
  final_res = pd.concat(res_list, keys=epochs_list)
  return final_res       


In [10]:
# molnet_bace
molnet_bace1_random = torch_mlp_reg('molnet_bace1_train_RandomSplitter.csv', 'molnet_bace1_test_RandomSplitter.csv')
molnet_bace1_scaffold = torch_mlp_reg('molnet_bace1_train_ScaffoldSplitter.csv', 'molnet_bace1_test_ScaffoldSplitter.csv')
molnet_bace1_butina = torch_mlp_reg('molnet_bace1_train_ButinaSplitter.csv', 'molnet_bace1_test_ButinaSplitter.csv')
molnet_bace1_df = pd.concat([molnet_bace1_random, molnet_bace1_scaffold, molnet_bace1_butina], keys = ['molnet_bace_random', 'molnet_bace_scaffold', 'molnet_bace_butina'])


In [11]:
molnet_bace1_df.to_csv('test.csv')

In [14]:
class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2215, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 2) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = sigmoid(self.fc4(h3))
        return output



def torch_mlp_reg(train_file, test_file):
  # process train data
  train_data = pd.read_csv(train_file)
  train_data_x = train_data['canonical_smiles']
  train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
  train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
  train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)

  train_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in train_mols]
  train_maccs_fps_array = np.asarray(train_maccs_fps, dtype=float)

  train_fps_array = np.concatenate([train_morgan_fps_array, train_maccs_fps_array],axis=1)



  train_x_tensor = torch.from_numpy(train_fps_array)
  train_y = train_data['Classification']
  train_y_tensor = torch.from_numpy(np.array(train_y))

  # process test data
  test_data = pd.read_csv(test_file)
  test_data_x = test_data['canonical_smiles']
  test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
  test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
  test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)

  test_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in test_mols]
  test_maccs_fps_array = np.asarray(test_maccs_fps, dtype=float)

  test_fps_array = np.concatenate([test_morgan_fps_array, test_maccs_fps_array],axis=1)


  test_x_tensor = torch.from_numpy(test_fps_array)
  test_y = test_data['Classification']
  test_y_tensor = torch.from_numpy(np.array(test_y))
  # model training
  epochs_list = [50, 80, 100, 130, 150, 170, 190, 200, 210, 230, 250]
  res_list = []
  for epochs in epochs_list:   
    #criterion = nn.MSELoss()
    model = QSAR_mlp()
    #losses = []
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    for epoch in range(epochs):
      data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).long()
      optimizer.zero_grad()
      y_pred = model(data)
      loss = F.cross_entropy(y_pred, target)
      #loss = criterion(y_pred.reshape(-1), target)
      #losses.append(loss.item())
      #if epoch % 20 == 0:
        #print("Loss {}: {}".format(epoch, loss.item()))
      loss.backward()
      optimizer.step()

    pred_test_y_prob = model(Variable(test_x_tensor).float()).tolist()

    torch_test_pred_list = []
    for i, test_prob in enumerate(pred_test_y_prob):
        pred_pos_prob = test_prob[1]
        torch_test_pred_list.append(round(pred_pos_prob, 3))

    torch_test_auc = round(roc_auc_score(test_y, torch_test_pred_list),2)
    torch_test_acc = round(accuracy_score(test_y, np.round(torch_test_pred_list)),2)
    torch_test_sen = round(recall_score(test_y, np.round(torch_test_pred_list)),2)
    torch_test_spc = round((torch_test_acc * len(test_y)- torch_test_sen * test_y.sum())/(len(test_y)-test_y.sum()),2)

    pred_train_y_prob = model(Variable(train_x_tensor).float()).tolist()

    torch_pred_list = []
    for i, train_prob in enumerate(pred_train_y_prob):
        pred_pos_prob = train_prob[1]
        torch_pred_list.append(round(pred_pos_prob, 3))

    torch_train_auc = round(roc_auc_score(train_y, torch_pred_list),2)
    torch_train_acc = round(accuracy_score(train_y, np.round(torch_pred_list)),2)
    torch_train_sen = round(recall_score(train_y, np.round(torch_pred_list)),2)
    torch_train_spc = round((torch_train_acc * len(train_y)- torch_train_sen * train_y.sum())/(len(train_y)-train_y.sum()),2)
    torch_perf = {'AUC':[torch_train_auc, torch_test_auc],
                'ACC':[torch_train_acc, torch_test_acc],
                'SEN':[torch_train_sen, torch_test_sen],
                'SPC':[torch_train_spc, torch_test_spc]}
    result = pd.DataFrame(torch_perf, index=['train', 'test'])
    res_list.append(result)
  final_res = pd.concat(res_list, keys=epochs_list)
  return final_res       

In [15]:
# chembl_bace
chembl_bace1_random = torch_mlp_reg('chembl_bace1_train_RandomSplitter.csv', 'chembl_bace1_test_RandomSplitter.csv')
chembl_bace1_scaffold = torch_mlp_reg('chembl_bace1_train_ScaffoldSplitter.csv', 'chembl_bace1_test_ScaffoldSplitter.csv')
chembl_bace1_butina = torch_mlp_reg('chembl_bace1_train_ButinaSplitter.csv', 'chembl_bace1_test_ButinaSplitter.csv')
chembl_bace1_df = pd.concat([chembl_bace1_random, chembl_bace1_scaffold, chembl_bace1_butina], keys = ['chembl_bace_random', 'chembl_bace_scaffold', 'chembl_bace_butina'])


In [16]:
# chembl_cdk2
chembl_cdk2_random = torch_mlp_reg('chembl_cdk2_train_RandomSplitter.csv', 'chembl_cdk2_test_RandomSplitter.csv')
chembl_cdk2_scaffold = torch_mlp_reg('chembl_cdk2_train_ScaffoldSplitter.csv', 'chembl_cdk2_test_ScaffoldSplitter.csv')
chembl_cdk2_butina = torch_mlp_reg('chembl_cdk2_train_ButinaSplitter.csv', 'chembl_cdk2_test_ButinaSplitter.csv')
chembl_cdk2_df = pd.concat([chembl_cdk2_random, chembl_cdk2_scaffold, chembl_cdk2_butina], keys = ['chembl_cdk2_random', 'chembl_cdk2_scaffold', 'chembl_cdk2_butina'])


In [17]:
# chembl_ach
chembl_ach_random = torch_mlp_reg('chembl_ach_train_RandomSplitter.csv', 'chembl_ach_test_RandomSplitter.csv')
chembl_ach_scaffold = torch_mlp_reg('chembl_ach_train_ScaffoldSplitter.csv', 'chembl_ach_test_ScaffoldSplitter.csv')
chembl_ach_butina = torch_mlp_reg('chembl_ach_train_ButinaSplitter.csv', 'chembl_ach_test_ButinaSplitter.csv')
chembl_ach_df = pd.concat([chembl_ach_random, chembl_ach_scaffold, chembl_ach_butina], keys = ['chembl_ach_random', 'chembl_ach_scaffold', 'chembl_ach_butina'])


In [18]:
with pd.ExcelWriter('four_datasets_DL_mlp_cls.xlsx') as writer:  
    molnet_bace1_df.to_excel(writer, sheet_name='molnet_bace')
    chembl_bace1_df.to_excel(writer, sheet_name='chembl_bace')
    chembl_cdk2_df.to_excel(writer, sheet_name='chembl_cdk2')
    chembl_ach_df.to_excel(writer, sheet_name='chembl_ach')